In [1]:
import gymnasium as gym

env = gym.make("room_env:RoomEnv-v1")
(observation, question), info = env.reset()
rewards = 0

while True:
    # There is one different thing in the RoomEnv from the original AAAI-2023 paper:
    # The reward is either +1 or -1, instead of +1 or 0.
    (observation, question), reward, done, truncated, info = env.step("my answer!")
    print((observation, question), reward, done, truncated, info)
    rewards += reward
    if done:
        break

# print(rewards)


/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/envs/registration.py:578: UserWarning: WARN: The environment RoomEnv-v1 is out of date. You should consider upgrading to version `v2`.
  logger.warn(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/envs/registration.py:435: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes']
  logger.warn(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:190: UserWarning: WARN: Future gymnasium versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:203: UserWarning: WARN: Future gymnasium versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to 

({'human': 'Amanda', 'object': 'bicycle', 'object_location': 'countryside', 'current_time': 1}, {'human': 'Amanda', 'object': 'bicycle'}) -1 False False {}
({'human': 'Ann', 'object': 'bowl', 'object_location': 'cupboard', 'current_time': 2}, {'human': 'Alexander', 'object': 'keyboard'}) -1 False False {}
({'human': 'Anthony', 'object': 'sheep', 'object_location': 'farm', 'current_time': 3}, {'human': 'Anthony', 'object': 'sheep'}) -1 False False {}
({'human': 'Arthur', 'object': 'donut', 'object_location': 'bakery', 'current_time': 4}, {'human': 'Alexander', 'object': 'keyboard'}) -1 False False {}
({'human': 'Barbara', 'object': 'donut', 'object_location': 'lap', 'current_time': 5}, {'human': 'Anthony', 'object': 'sheep'}) -1 False False {}
({'human': 'Beverly', 'object': 'train', 'object_location': 'zoo', 'current_time': 6}, {'human': 'Arthur', 'object': 'donut'}) -1 False False {}
({'human': 'Billy', 'object': 'bird', 'object_location': 'tree', 'current_time': 7}, {'human': 'Anthon

In [2]:
(observation, question), reward, done, truncated, info


(({'human': 'Alexander',
   'object': 'dog',
   'object_location': 'kennel',
   'current_time': 128},
  None),
 -1,
 True,
 False,
 {})